# 🧠 CRM PCS Pipeline Debug
Full lifecycle audit of CRM straddle across parsing, strategy tagging, freeze, and scoring logic.

In [2]:
from core.phase1_clean import phase1_load_and_clean_raw_v2

# Load raw options data
df, snapshot_path = phase1_load_and_clean_raw_v2(
    input_path="/Users/haniabadi/Documents/Windows/Positions_Account_.csv",
    snapshot_dir="/Users/haniabadi/Documents/Windows/Optionrec/drift"
)


⏳ Loading file: /Users/haniabadi/Documents/Windows/Positions_Account_.csv
🟢 File loaded!
🧹 Deduplicated 0 rows (Symbol only — fallback)
✅ Phase 1 complete.
📊 Rows: 7, Columns: 38
💾 Saved snapshot: /Users/haniabadi/Documents/Windows/Optionrec/drift/master_with_drift_2025-07-28_17-10-41.csv
⏱ Duration: 0.02s


In [3]:
from core.phase2_parse import phase2_parse_symbols, phase21_strategy_tagging
from core.phase3_pcs_score import calculate_pcs, calculate_ivhv_gap, calculate_skew_and_kurtosis

df = phase2_parse_symbols(df)
df = phase21_strategy_tagging(df)
df = calculate_ivhv_gap(df)
df = calculate_skew_and_kurtosis(df)
df = calculate_pcs(df)

df[["TradeID", "PCS", "PCS_Entry", "Vega", "Vega_Entry"]]


✅ Phase 2.0 complete: OCC symbols parsed
✅ Phase 2.1 complete: Strategy tagging with TradeID-only logic


/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period=period, interval='1d', progress=False)
/Users/haniabadi/Documents/Github/options/core/phase3_pcs_score.py:11: FutureWarning: YF.download() has 

📉 Skew nulls: 7 | TradeIDs: 7
🩹 Applied fallback: Skew/Kurtosis set to 0 where missing


KeyError: "['PCS_Entry'] not in index"

In [4]:
from core.phase3_5_freeze_fields import phase3_5_fill_freeze_fields

df = phase3_5_fill_freeze_fields(df)

df[["TradeID", "PCS", "PCS_Entry", "Vega", "Vega_Entry"]]


📌 Phase 3.5: Full Freeze with Greeks, Capital, Tags, IVHV, Earnings, Multi-leg Logic...


KeyError: 'Premium'

In [ ]:
import sys
sys.path.append("core")

from core.pcs_engine_v3_unified import pcs_engine_v3_2_strategy_aware
import pandas as pd

DEBUG_ID = "CAT250801C410"
DEBUG_PATH = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"

# Load master
df_master = pd.read_csv(DEBUG_PATH)

# Run PCS Engine
df_result = pcs_engine_v3_2_strategy_aware(df_master)

# Add PCS_Live field for debug compatibility
df_result["PCS_Live"] = df_result["PCS_UnifiedScore"]

# Extract just this TradeID
df_trade = df_result[df_result["TradeID"] == DEBUG_ID]
df_trade.T  # vertical output


In [ ]:
import pandas as pd
df = pd.read_csv(DEBUG_PATH)

# Show rows matching TradeID
df_trade = df[df["TradeID"] == DEBUG_ID]
print(f"✅ Loaded {len(df_trade)} rows for TradeID: {DEBUG_ID}")
df_trade.head(1).T  # vertical debug


In [ ]:
print("📌 Entry Snapshot:")
print("PCS_Entry:", df_trade["PCS_Entry"].iloc[0])
print("Strategy:", df_trade["Strategy"].iloc[0])
print("Vega_Entry:", df_trade["Vega_Entry"].iloc[0])
print("BreakEven:", df_trade["BreakEven"].iloc[0])


In [ ]:
# Assign PCS_Live BEFORE filtering
df_result["PCS_Live"] = df_result["PCS_UnifiedScore"]

# Then refilter
df_trade = df_result[df_result["TradeID"] == DEBUG_ID]

# Now this will work
print("PCS_Live:", df_trade["PCS_Live"].iloc[0])


print("📉 Exit Metrics:")
print("Held_ROI:", df_trade["Held_ROI%"].iloc[0])
print("PCS_Live:", df_trade["PCS_Live"].iloc[0])
print("Rec_Action:", df_trade["Rec_Action"].iloc[0])
print("OutcomeTag:", df_trade["OutcomeTag"].iloc[0] if "OutcomeTag" in df_trade.columns else "NA")
print("Success_Prob:", df_trade["Success_Prob"].iloc[0] if "Success_Prob" in df_trade.columns else "NA")




In [ ]:
from core.phase7_drift_engine import run_phase7_drift_engine
from rec_engine_v6_overlay import run_v6_overlay

drift_dir = "/Users/haniabadi/Documents/Windows/Optionrec/drift"

# ✅ Rerun Drift Engine
df_drifted = run_phase7_drift_engine(drift_dir=drift_dir, update_master=False)

# ✅ Reload master + run V6 overlay
df_master = pd.read_csv(DEBUG_PATH)
df_overlayed = run_v6_overlay(master_path=DEBUG_PATH)
df_overlayed[df_overlayed["TradeID"] == DEBUG_ID].T


In [ ]:
df_overlayed[df_overlayed["TradeID"] == DEBUG_ID][[
    "Rec_V6", "Rec_Tier", "Rec_Action", "Confidence", 
    "PCS_UnifiedScore", "PCS_Entry", 
    "Chart_CompositeScore", "Trim_Trigger", "PCS_Drift"
]].T


In [ ]:
import matplotlib.pyplot as plt
plt.barh(["PCS", "PCS_UnifiedScore", "PCS_SignalScore"], [
    df_trade["PCS"].iloc[0],
    df_trade["PCS_UnifiedScore"].iloc[0],
    df_trade["PCS_SignalScore"].iloc[0]
])
plt.title(f"PCS Layers for {DEBUG_ID}")
plt.show()


In [ ]:
def evaluate_trade_logic(row):
    lines = []
    if row["PCS_Entry"] >= 75 and row["Held_ROI%"] < 0:
        lines.append("❌ False GEM")
    elif row["PCS_Entry"] < 70 and row["Held_ROI%"] > 50:
        lines.append("❓ Missed Opportunity")
    if row["PCS_Drift"] > 15:
        lines.append("⚠️ Drift Trigger")
    if row["Success_Prob"] < 0.6:
        lines.append("🤔 Model was cautious")
    if row["Rec_Action"] == "EXIT" and ROI > 50:
        lines.append("📉 Premature exit?")
    return " | ".join(lines)

print("🧠 Evaluation:", evaluate_trade_logic(df_trade.iloc[0]))


In [ ]:
# Load original snapshot
snap = pd.read_csv("/Users/haniabadi/Documents/Windows/Optionrec/drift/positions_2025-07-25_10-40-31.csv")
snap_trade = snap[snap["TradeID"] == DEBUG_ID]
snap_trade.T


In [ ]:
# Key PCS and strategy lifecycle fields
cols = [
    "Symbol", "Underlying", "OptionType", "Strike", "Expiration", "DTE", "TradeID", "Strategy", "Strategy_Tier",
    "PCS", "PCS_Entry", "PCS_UnifiedScore", "PCS_PersonaScore", "PCS_Tier",
    "Confidence Tier", "Confidence_Entry", "Gamma", "Vega", "Delta", "Theta",
    "IV Mid", "HV", "IVHV_Gap_Entry", "Chart_Trend", "Rec_Action", "Rec_Rationale",
    "Tag_Intent", "Structure", "Tag_EdgeType", "Held_ROI%", "Trade_Health_Tier"
]
crm_debug = crm[cols]
crm_debug


In [ ]:

# core/phase3_pcs_score.py (patched)

import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import skew, kurtosis
import yfinance as yf

# === IV–HV Gap Calculation ===
def calculate_ivhv_gap(df: pd.DataFrame) -> pd.DataFrame:
    def fetch_hv(symbol, period="30d"):
        try:
            data = yf.download(symbol, period=period, interval='1d', progress=False)
            if 'Close' not in data.columns:
                return np.nan
            data['Returns'] = data['Close'].pct_change()
            return data['Returns'].std() * np.sqrt(252)
        except Exception as e:
            print(f"⚠️ Error fetching HV for {symbol}: {e}")
            return np.nan

    if "HV" not in df.columns or df["HV"].isnull().all():
        df['HV'] = df['Underlying'].apply(fetch_hv)

    df['IV Mid'] = pd.to_numeric(df['IV Mid'], errors='coerce')
    df['HV'] = pd.to_numeric(df['HV'], errors='coerce')
    df['IVHV_Gap_Entry'] = df['IV Mid'] - df['HV']
    return df

# === Skew & Kurtosis Calculation ===
def calculate_skew_and_kurtosis(df: pd.DataFrame) -> pd.DataFrame:
    if "GroupKey" in df.columns:
        def safe_skew(x):
            x_clean = x.dropna()
            return skew(x_clean) if len(x_clean) > 1 else np.nan

        def safe_kurt(x):
            x_clean = x.dropna()
            return kurtosis(x_clean) if len(x_clean) > 1 else np.nan

        df["Skew_Entry"] = df.groupby("GroupKey")["IV Mid"].transform(safe_skew)
        df["Kurtosis_Entry"] = df.groupby("GroupKey")["IV Mid"].transform(safe_kurt)

        print(f"📉 Skew nulls: {df['Skew_Entry'].isnull().sum()} | GroupKey count: {df['GroupKey'].nunique()}")
    else:
        val_skew = skew(df["IV Mid"].dropna())
        val_kurt = kurtosis(df["IV Mid"].dropna())
        df["Skew_Entry"] = val_skew
        df["Kurtosis_Entry"] = val_kurt

    # 🩹 Fallback for directional or small-group trades
    df["Skew_Entry"] = df["Skew_Entry"].fillna(0.0)
    df["Kurtosis_Entry"] = df["Kurtosis_Entry"].fillna(0.0)
    print("🩹 Applied fallback: Skew/Kurtosis set to 0 where missing")
    return df

# === PCS Scoring ===
def calculate_pcs(df: pd.DataFrame) -> pd.DataFrame:
    # Ensure IVHV Gap is present
    if "IVHV_Gap_Entry" not in df.columns or df["IVHV_Gap_Entry"].isnull().all():
        df = calculate_ivhv_gap(df)

    def calculate_row_subscores(row):
        gamma = np.nan_to_num(pd.to_numeric(row.get("Gamma", 0)), nan=0)
        vega = np.nan_to_num(pd.to_numeric(row.get("Vega", 0)), nan=0)
        roi = np.nan_to_num(pd.to_numeric(row.get("% Total G/L", 0)), nan=0)
        ivhv_gap = np.nan_to_num(pd.to_numeric(row.get("IVHV_Gap_Entry", 0)), nan=0)

        gamma_score = min(gamma * 1000, 25)
        vega_score = min(vega * 80, 25)
        roi_score = 15 if roi >= 3.0 else 10 if roi >= 2.0 else 5
        ivhv_score = 5 if ivhv_gap >= 5.0 else 0

        return pd.Series([gamma_score, vega_score, roi_score, ivhv_score])

    today = pd.to_datetime(datetime.now().date())
    if "Expiration" in df.columns:
        df['DTE'] = (pd.to_datetime(df['Expiration']) - today).dt.days

    df[["PCS_GammaScore", "PCS_VegaScore", "PCS_ROIScore", "PCS_IVHVBonus"]] = df.apply(
        calculate_row_subscores, axis=1
    )
    df["PCS"] = df[["PCS_GammaScore", "PCS_VegaScore", "PCS_ROIScore", "PCS_IVHVBonus"]].sum(axis=1)

    if "GroupKey" in df.columns:
        df["PCS_GroupAvg"] = df.groupby("GroupKey")["PCS"].transform("mean")

    df["PCS_Tier"] = df["PCS"].apply(
        lambda pcs: "Tier 1" if pcs >= 80 else "Tier 2" if pcs >= 70 else "Tier 3" if pcs >= 60 else "Tier 4"
    )

    df["Needs_Revalidation"] = (df["PCS"] < 65) | (df["Vega"] < 0.25)
    df["PCS_Drift"] = df["PCS"] - df.get("PCS_Entry", df["PCS"])
    df["Vega_ROC"] = df["Vega"] - df.get("Vega_Entry", df["Vega"])
    df["IV_Drift"] = df["IV Mid"] - df.get("IV_Entry", df["IV Mid"])

    return df


In [ ]:
# 🔍 Quick PCS Debug for CRM (no module needed)

import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf

# Load data
df = pd.read_csv("active_master.csv")
crm = df[df["Underlying"] == "CRM"].copy()

# HV fetch if missing
def fetch_hv(symbol):
    data = yf.download(symbol, period="30d", interval='1d', progress=False)
    if 'Close' not in data.columns:
        return np.nan
    data['Returns'] = data['Close'].pct_change()
    return data['Returns'].std() * np.sqrt(252)

if "HV" not in crm.columns or crm["HV"].isnull().all():
    crm["HV"] = crm["Underlying"].apply(fetch_hv)

crm["IVHV_Gap_Entry"] = pd.to_numeric(crm["IV Mid"], errors='coerce') - pd.to_numeric(crm["HV"], errors='coerce')

# Recalculate PCS components
def score(row):
    gamma_score = min(row["Gamma"] * 1000, 25)
    vega_score = min(row["Vega"] * 80, 25)
    roi_score = 15 if row["% Total G/L"] >= 3 else 10 if row["% Total G/L"] >= 2 else 5
    ivhv_score = 5 if row["IVHV_Gap_Entry"] >= 5 else 0
    total = gamma_score + vega_score + roi_score + ivhv_score
    print(f"{row['Symbol']} → Gamma: {gamma_score}, Vega: {vega_score}, ROI: {roi_score}, IVHV Bonus: {ivhv_score} → PCS: {total}")
    return total

crm["PCS_Debug"] = crm.apply(score, axis=1)
crm[["Symbol", "OptionType", "Gamma", "Vega", "% Total G/L", "IV Mid", "HV", "IVHV_Gap_Entry", "PCS_Debug"]]


### 🔍 Observations:
- OptionType shows both `CALL` and `PUT` legs ✅
- But TradeIDs are different ❌
- Strategy is 'Unclassified' → no persona logic applied ❌
- PCS_PersonaScore = 0 → Strategy_Match = Violated → Rec_Action = EXIT ❌
- Chart Trend and Vol Edge were correctly detected ✅

---

### 🔧 What To Fix:
1. Phase 2.1 should detect both legs and group them
2. Assign a shared `TradeID` (e.g., CRM20250822_Straddle)
3. Set Strategy = "Straddle", Strategy_Tier = Tier2_Neutral
4. Re-run PCS scoring for correct persona fit


In [ ]:
# Check grouping problem
crm[["Symbol", "OptionType", "Strike", "Expiration", "TradeID", "Strategy"]]

In [ ]:
# Optional fix: force strategy for test
df.loc[df["Underlying"] == "CRM", "Strategy"] = "Straddle"
df.loc[df["Underlying"] == "CRM", "Strategy_Tier"] = "Tier2_Neutral"
df[df["Underlying"] == "CRM"][["Strategy", "Strategy_Tier"]].drop_duplicates()

In [ ]:
DEBUG_ID = "CAT250801C410"  # or a known TradeID
DEBUG_PATH = "/Users/haniabadi/Documents/Windows/Optionrec/active_master.csv"
